# Heroes of Pymoli Data Analysis

* The most profitable age group for the company is from 20 to 24 years but the average purchase total is higher among people between 30 and 34 years old
* There are 4.5x more male players than female players but they only outspend them by ~4x
* The most expensive item does not make the cut of 5 most profitable items

Dependencies

In [1]:
import os
import pandas as pd

Data &rarr; data frame

In [2]:
purchase_data_path = os.path.join('Resources','purchase_data.json')
purchase_df = pd.read_json(purchase_data_path)

#### Get total number of players

In [3]:
#Distinguish players by SN attribute
total_players = len(purchase_df['SN'].unique())
total_players_df = pd.DataFrame({'Total Players':[total_players]})
total_players_df


,Total Players
0,573


####  Do purchase analysis

In [4]:
total_items = len(purchase_df['Item ID'].unique())
average_item_price = purchase_df['Price'].mean()
total_purchases = purchase_df['Price'].count()
total_revenue = purchase_df['Price'].sum()
#create dataframe with computed values
purchase_analysis_total_df = pd.DataFrame(
    {'Number of Unique Items':[total_items],
     'Average Purchase Price':[average_item_price],
     'Total Number of Purchases':[total_purchases],
     'Total Revenue':[total_revenue]})
#format currency
purchase_analysis_total_df['Average Purchase Price'] = \
purchase_analysis_total_df['Average Purchase Price'].map('${:,.2f}'.format)
purchase_analysis_total_df['Total Revenue'] = \
purchase_analysis_total_df['Total Revenue'].map('${:,.2f}'.format)
#re-arrange columns
purchase_analysis_total_df = purchase_analysis_total_df[['Number of Unique Items',
                                                        'Average Purchase Price',
                                                        'Total Number of Purchases',
                                                        'Total Revenue']]
purchase_analysis_total_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


#### Gender demographics

In [5]:
males_df = purchase_df.loc[purchase_df['Gender'] == 'Male']
females_df = purchase_df.loc[purchase_df['Gender'] == 'Female']
#use SN to distinguish players and avoid double counting
males = len(males_df['SN'].unique())
females = len(females_df['SN'].unique())
others = len(purchase_df['SN'].unique()) - males - females
#create dataframe 
gender_df = pd.DataFrame({'Male':[100*males/total_players, males],
                          'Female':[100*females/total_players, females],
                         'Other / Non-Disclose':[100*others/total_players, others]})
#re-arrange columns
gender_df = gender_df[['Male', 'Female','Other / Non-Disclose']]
gender_df.index = ['Percentage of Players', 'Total Count']
gender_df = gender_df.transpose()
#format percent and integers
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map('{:,.2f}%'.format)
gender_df['Total Count'] = gender_df['Total Count'].map('{:,.0f}'.format)
gender_df

,Percentage of Players,Total Count
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclose,1.40%,8


#### Do purchase analysis by gender

In [6]:
#group data into gender categories
purchase_gender_gp = purchase_df.groupby('Gender')
#aggregate to get each category's count, sum, mean, divide sum/count to get norm totals
gender_count_series =  purchase_gender_gp['Price'].count()
gender_mean_series = purchase_gender_gp['Price'].mean().map('${:,.3f}'.format)
gender_total_series = purchase_gender_gp['Price'].sum().map('${:,.3f}'.format)
gender_norm_series = \
purchase_gender_gp['Price'].sum()/purchase_gender_gp['Price'].count()
gender_norm_series = gender_norm_series.map('${:,.3f}'.format)
#merge series to create dataframe
purchase_gender_df = pd.concat([gender_count_series, gender_mean_series,
                                gender_total_series, gender_norm_series], axis=1)
purchase_gender_df.columns = ['Purchase Count', 'Average Purchase Price',
                              'Total Purchase Value', 'Normalized Totals']
purchase_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.816,$382.910,$2.816
Male,633,$2.951,"$1,867.680",$2.951
Other / Non-Disclosed,11,$3.249,$35.740,$3.249


#### Age Demographics

In [7]:
#define age bins for analysis
age_bins = [0, 10, 15, 20, 25, 30, 35, 40,45]
labels = ['<10','10-14','15-19', '20-24', '25-29', '30-34', '35-39', '>40']
#assign rows to their corresponding age bins generated by cut
purchase_df['Age Range'] = pd.cut(purchase_df['Age'], bins=age_bins, labels=labels)
#group by age bins
purchase_age_range_gp = purchase_df.groupby('Age Range')
#aggregate per bin information
purchase_age_count_series = purchase_age_range_gp['Price'].count()
purchase_age_mean_series = purchase_age_range_gp['Price'].mean().map('${:,.3f}'.format)
purchase_age_total_series = purchase_age_range_gp['Price'].sum().map('${:,.3f}'.format)
purchase_age_norm_series = \
purchase_age_range_gp['Price'].sum()/purchase_age_range_gp['Price'].count()
purchase_age_norm_series = purchase_age_norm_series.map('${:,.3f}'.format)
#merge series to create dataframe
purchase_age_df = pd.concat([purchase_age_count_series, purchase_age_mean_series,
                             purchase_age_total_series, purchase_age_norm_series], axis=1)
purchase_age_df.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value',
                           'Normalized Totals']
purchase_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
<10,32,$3.019,$96.620,$3.019
10-14,78,$2.874,$224.150,$2.874
15-19,184,$2.874,$528.740,$2.874
20-24,305,$2.959,$902.610,$2.959
25-29,76,$2.892,$219.820,$2.892
30-34,58,$3.073,$178.260,$3.073
35-39,44,$2.898,$127.490,$2.898
>40,3,$2.880,$8.640,$2.880


#### Top Spenders

In [8]:
#group by SN to get unique customers
purchase_sn_gp = purchase_df.groupby('SN')
#aggregate per user info
#we will defer formating totals since we need to sort their values
purchase_sn_total_series = purchase_sn_gp['Price'].sum()
purchase_sn_count_series = purchase_sn_gp['SN'].count()
purchase_sn_mean_series = purchase_sn_gp['Price'].mean().map('${:,.3f}'.format)
#merge series to create dataframe
purchase_sn_df = pd.concat([purchase_sn_count_series, purchase_sn_mean_series, purchase_sn_total_series], axis=1)
purchase_sn_df.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
#find top spenders by sorting
top_spenders = purchase_sn_df.sort_values(by='Total Purchase Value', ascending=False).head(5)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:,.3f}'.format)
top_spenders



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.412,$17.060
Saedue76,4,$3.390,$13.560
Mindimnya67,4,$3.185,$12.740
Haellysu29,3,$4.243,$12.730
Eoda93,3,$3.860,$11.580


#### Most Popular Items

In [9]:
#group items by Item ID
purchase_id_gp = purchase_df.groupby(['Item ID','Item Name'])
#aggregate per item info
purchase_id_count = purchase_id_gp['Item ID'].count()
#we will defer formating totals since we need to sort their values
purchase_id_total = purchase_id_gp['Price'].sum()
purchase_id_price = purchase_id_gp['Price'].mean().map('${:,.3f}'.format)
#concatenate series into a datafame
purchase_id_df =pd.concat([purchase_id_count, purchase_id_price, purchase_id_total], axis=1)
purchase_id_df.columns = ['Purchase Count', 'Item Price', 'Total Purchase Value']
most_popular = purchase_id_df.sort_values(by='Purchase Count', ascending=False).head(5)
most_popular

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.350,25.85
84,Arcane Gem,11,$2.230,24.53
31,Trickster,9,$2.070,18.63
175,Woeful Adamantite Claymore,9,$1.240,11.16
13,Serenity,9,$1.490,13.41


#### Most Profitable Items

In [10]:
most_profitable = purchase_id_df.sort_values(by='Total Purchase Value', ascending=False)
most_profitable['Total Purchase Value'] = most_profitable['Total Purchase Value'].map('${:,.3f}'.format)
most_profitable.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.140,$37.260
115,Spectral Diamond Doomblade,7,$4.250,$29.750
32,Orenmir,6,$4.950,$29.700
103,Singed Scalpel,6,$4.870,$29.220
107,"Splitter, Foe Of Subtlety",8,$3.610,$28.880
